# Scheinkorrelation

Das Konzept der Korrelation bezeichnet, dass zwischen zwei Attributen eine Kopplung besteht.
Eine positive Korrelation bedeutet, dass zwei Dinge gemeinsam auftreten, z. B. bedeutet viel Autoverkehr in den Innenstädten eine höhere Schadstoffbelastung.
Eine negative Korrelation bedeutet, dass die Kopplung umgekehrt ist.
Je mehr Menschen mit dem Fahrrad fahren, desto weniger Autos sind in einer Stadt unterwegs.

Eine Scheinkorrelation (Englisch: *spurious correlation*) bezeichnet den Fall, dass zwei Attribute rein zufällig miteinander korrelieren.
In diesem Notebook wird gezeigt, weswegen Scheinkorrelationen für das Maschinelle Lernen eine Herausforderung darstellen.

In [5]:
import random
import pandas as pd

Nun erstellen wir uns einen Datensatz mit 5 verschiedenen Attributen.
In diesem Beispiel sind es, wie im Tennisdatensatz, Wetterbeobachtungen.
Der Datensatz hat insgesamt 20 Reihen.

In [6]:
# Fixiere Zufallsgenerator
random.seed(0)

number_rows = 20

df = pd.DataFrame({
    "Outlook": [random.choice(["Sunny", "Overcast", "Rain"]) for _ in range(number_rows)],
    "Temperature": [random.choice(["Hot", "Mild", "Cool"]) for _ in range(number_rows)],
    "Humidity": [random.choice(["High", "Normal"]) for _ in range(number_rows)],
    "Wind": [random.choice(["Weak", "Strong"]) for _ in range(number_rows)],
    "Play Tennis?": [random.choice(["Yes", "No"]) for _ in range(number_rows)],
})

df

,Outlook,Temperature,Humidity,Wind,Play Tennis?
0,Overcast,Cool,Normal,Strong,Yes
1,Overcast,Cool,Normal,Weak,Yes
2,Sunny,Hot,High,Strong,Yes
3,Overcast,Mild,High,Strong,Yes
4,Rain,Hot,High,Weak,Yes
5,Overcast,Cool,Normal,Strong,Yes
6,Overcast,Hot,High,Weak,No
7,Overcast,Cool,Normal,Strong,No
8,Overcast,Mild,Normal,Strong,Yes
9,Overcast,Mild,High,Weak,Yes


Für weitere Berechnungen werden die Kategorien in numerische Werte übersetzt.
Hier verwenden wir nicht das One-Hot-Encoding, sondern enkodieren jede mögliche Ausprägung mit einem Zahlenwert.

In [7]:
df_cat = df.assign(**{
    col: df[col].astype('category').cat.codes for col in ["Humidity", "Wind", "Play Tennis?"]
})
df_cat = df_cat.assign(
    Outlook=df["Outlook"].astype(
        pd.CategoricalDtype(categories=["Rain", "Overcast", "Sunny"], ordered=True)).cat.codes,
    Temperature=df["Temperature"].astype(
        pd.CategoricalDtype(categories=["Cool", "Mild", "Hot"], ordered=True)).cat.codes
)
df_cat

,Outlook,Temperature,Humidity,Wind,Play Tennis?
0,1,0,1,0,1
1,1,0,1,1,1
2,2,2,0,0,1
3,1,1,0,0,1
4,0,2,0,1,1
5,1,0,1,0,1
6,1,2,0,1,0
7,1,0,1,0,0
8,1,1,1,0,1
9,1,1,0,1,1


Wir haben gerade gesehen, es wird die Bibliothek `random` benutzt, also sind sämtliche Attribute zufällig gewählt worden.
Es darf also keinen Zusammenhang zwischen dem Wetter und der Frage geben, ob der Nachbar Tennis spielt!

In [8]:
import sklearn.tree
dt = sklearn.tree.DecisionTreeClassifier()
eingabe = df_cat[["Outlook", "Temperature", "Humidity", "Wind"]]
ziel = df_cat[["Play Tennis?"]]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(eingabe, ziel, random_state=6)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

0.8

Wir haben alles richtig gemacht:
Wir haben mit den Trainings-Daten trainiert und mit den Test-Daten getestet.
Irgendwas ist dennoch falsch gelaufen.

<span style="color:blue; font-weight:bold">Aufgabe 1</span>

Inspizieren Sie den Code und finden Sie eine Möglichkeit, das Problem zu beheben.
Wie kann verhindert werden, dass der Entscheidungsbaum aus den Zufallsdaten scheinbar doch eine Prognose findet?
Passen Sie gerne den Code direkt an und markieren sich mit Kommentaren, was Sie geändert haben.

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.